In [2]:
#importing modules
import pandas as pd
import os
import sklearn
import numpy as np
import pandas as pd
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, metrics
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from google.colab import drive
import joblib

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


DATA PREPOROCESSING

In [3]:
#loading the dataset for preproccesing and feature engineering
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_21.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 15.9+ MB


In [4]:
#finding the correlation between each feature and the overall rating of the player
#finding the correlation between each physical attribute and the overall rating of the player is our focus, rather than using extrinsic features such as wages and club name
selected_features = ['overall', 'potential', 'age', 'height_cm', 'weight_kg', 'pace', 'shooting', 'passing',
                     'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
                     'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys',
                     'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
                     'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
                     'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina',
                     'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
                     'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure',
                     'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle',
                     'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning',
                     'goalkeeping_reflexes', 'goalkeeping_speed']
correlation_matrix = df[selected_features].corr()
#sorts the features with the heighest correlation to the overall in ascending order
correlation_matrix['overall'].sort_values(ascending=False)

overall                        1.000000
movement_reactions             0.867234
passing                        0.711521
mentality_composure            0.705252
dribbling                      0.641097
potential                      0.636366
power_shot_power               0.558372
physic                         0.530386
mentality_vision               0.509087
attacking_short_passing        0.502191
goalkeeping_speed              0.497350
shooting                       0.488315
skill_long_passing             0.487147
age                            0.468197
skill_ball_control             0.449372
skill_curve                    0.420495
attacking_crossing             0.410530
power_long_shots               0.407525
mentality_aggression           0.401920
skill_fk_accuracy              0.385617
power_stamina                  0.381869
skill_dribbling                0.378455
attacking_volleys              0.374330
defending                      0.358524
power_strength                 0.358049


In [5]:
#selecting the best correlated features
selected_columns = ['skill_long_passing', 'skill_ball_control', 'skill_curve', 'skill_fk_accuracy', 'skill_dribbling',
                    'goalkeeping_handling', 'attacking_crossing', 'movement_reactions', 'age',
                    'mentality_aggression', 'mentality_vision', 'mentality_interceptions', 'goalkeeping_kicking',
                    'goalkeeping_speed', 'shooting', 'dribbling', 'power_shot_power','power_long_shots', 'defending', 'goalkeeping_reflexes',
                    'potential', 'mentality_composure', 'mentality_positioning', 'mentality_penalties', 'passing',
                    'attacking_short_passing', 'physic', 'overall', 'goalkeeping_positioning', 'goalkeeping_diving',
                    'attacking_volleys', 'attacking_finishing']


In [6]:
#loading the dataset with the best correlated features
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_21.csv', usecols=selected_columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  18944 non-null  int64  
 1   potential                18944 non-null  int64  
 2   age                      18944 non-null  int64  
 3   shooting                 16861 non-null  float64
 4   passing                  16861 non-null  float64
 5   dribbling                16861 non-null  float64
 6   defending                16861 non-null  float64
 7   physic                   16861 non-null  float64
 8   attacking_crossing       18944 non-null  int64  
 9   attacking_finishing      18944 non-null  int64  
 10  attacking_short_passing  18944 non-null  int64  
 11  attacking_volleys        18944 non-null  int64  
 12  skill_dribbling          18944 non-null  int64  
 13  skill_curve              18944 non-null  int64  
 14  skill_fk_accuracy     

In [7]:
df.head()


,overall,potential,age,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,...,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,93,93,33,92.0,91.0,95.0,38.0,65.0,85,95,...,93,95,75,96,6,11,15,14,8,NaN
1,92,92,35,93.0,81.0,89.0,35.0,77.0,84,95,...,95,82,84,95,7,11,15,14,11,NaN
2,91,91,31,91.0,78.0,85.0,43.0,82.0,71,94,...,94,79,88,88,15,6,12,8,10,NaN
3,91,91,28,85.0,86.0,94.0,36.0,59.0,85,87,...,87,90,92,93,9,9,15,15,11,NaN
4,91,91,29,86.0,93.0,88.0,64.0,78.0,94,82,...,88,94,84,91,15,13,5,10,13,NaN


In [8]:
# replacing nan values with 0
df.fillna(0, inplace=True)
# combine instances with overall ratings 92 and 93 into a single class '92'
df['overall'] = df['overall'].apply(lambda x: 92 if x in [92, 93] else x)


FEATURE ENGINEERING

In [9]:
# we combined multiply features with similar attributes to form one feature.
# this helped us decrease the mean absolute error of our model when testing

shooting_attributes = ['shooting', 'power_shot_power', 'power_long_shots', 'attacking_volleys','attacking_finishing']

df['shooting_skills'] = df[shooting_attributes].mean(axis=1)

df.drop(columns=shooting_attributes, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  18944 non-null  int64  
 1   potential                18944 non-null  int64  
 2   age                      18944 non-null  int64  
 3   passing                  18944 non-null  float64
 4   dribbling                18944 non-null  float64
 5   defending                18944 non-null  float64
 6   physic                   18944 non-null  float64
 7   attacking_crossing       18944 non-null  int64  
 8   attacking_short_passing  18944 non-null  int64  
 9   skill_dribbling          18944 non-null  int64  
 10  skill_curve              18944 non-null  int64  
 11  skill_fk_accuracy        18944 non-null  int64  
 12  skill_long_passing       18944 non-null  int64  
 13  skill_ball_control       18944 non-null  int64  
 14  movement_reactions    

In [10]:
# we combined multiply features with similar attributes to form one feature.
# this helped us decrease the mean absolute error of our model when testing
mentality_attributes = ['mentality_aggression', 'mentality_interceptions', 'mentality_positioning',
                        'mentality_vision', 'mentality_penalties', 'mentality_composure']
imputer = SimpleImputer(strategy='mean')
df[mentality_attributes] = imputer.fit_transform(df[mentality_attributes])

df['mentality'] = df[mentality_attributes].mean(axis=1)

df.drop(columns=mentality_attributes, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  18944 non-null  int64  
 1   potential                18944 non-null  int64  
 2   age                      18944 non-null  int64  
 3   passing                  18944 non-null  float64
 4   dribbling                18944 non-null  float64
 5   defending                18944 non-null  float64
 6   physic                   18944 non-null  float64
 7   attacking_crossing       18944 non-null  int64  
 8   attacking_short_passing  18944 non-null  int64  
 9   skill_dribbling          18944 non-null  int64  
 10  skill_curve              18944 non-null  int64  
 11  skill_fk_accuracy        18944 non-null  int64  
 12  skill_long_passing       18944 non-null  int64  
 13  skill_ball_control       18944 non-null  int64  
 14  movement_reactions    

In [11]:
# we combined multiply features with similar attributes to form one feature.
# this helped us decrease the mean absolute error of our model when testing

skill_attributes = ['skill_long_passing', 'skill_ball_control', 'skill_curve', 'skill_fk_accuracy', 'skill_dribbling']

df['technical_skills'] = df[skill_attributes].mean(axis=1)

df.drop(columns=skill_attributes, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  18944 non-null  int64  
 1   potential                18944 non-null  int64  
 2   age                      18944 non-null  int64  
 3   passing                  18944 non-null  float64
 4   dribbling                18944 non-null  float64
 5   defending                18944 non-null  float64
 6   physic                   18944 non-null  float64
 7   attacking_crossing       18944 non-null  int64  
 8   attacking_short_passing  18944 non-null  int64  
 9   movement_reactions       18944 non-null  int64  
 10  goalkeeping_diving       18944 non-null  int64  
 11  goalkeeping_handling     18944 non-null  int64  
 12  goalkeeping_kicking      18944 non-null  int64  
 13  goalkeeping_positioning  18944 non-null  int64  
 14  goalkeeping_reflexes  

In [12]:
# transform float values to int type for easier manipulation
df= df.astype(int)
# we combined multiply features with similar attributes to form one feature.
# this helped us decrease the mean absolute error of our model when testing

goalkeeping_attributes = ['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
                          'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed']

imputer = SimpleImputer(strategy='mean')
df[goalkeeping_attributes] = imputer.fit_transform(df[goalkeeping_attributes])

df['goalkeeping_ability'] = df[goalkeeping_attributes].mean(axis=1)

df.drop(columns=goalkeeping_attributes, inplace=True)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  18944 non-null  int64  
 1   potential                18944 non-null  int64  
 2   age                      18944 non-null  int64  
 3   passing                  18944 non-null  int64  
 4   dribbling                18944 non-null  int64  
 5   defending                18944 non-null  int64  
 6   physic                   18944 non-null  int64  
 7   attacking_crossing       18944 non-null  int64  
 8   attacking_short_passing  18944 non-null  int64  
 9   movement_reactions       18944 non-null  int64  
 10  shooting_skills          18944 non-null  int64  
 11  mentality                18944 non-null  int64  
 12  technical_skills         18944 non-null  int64  
 13  goalkeeping_ability      18944 non-null  float64
dtypes: float64(1), int64(1

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  18944 non-null  int64  
 1   potential                18944 non-null  int64  
 2   age                      18944 non-null  int64  
 3   passing                  18944 non-null  int64  
 4   dribbling                18944 non-null  int64  
 5   defending                18944 non-null  int64  
 6   physic                   18944 non-null  int64  
 7   attacking_crossing       18944 non-null  int64  
 8   attacking_short_passing  18944 non-null  int64  
 9   movement_reactions       18944 non-null  int64  
 10  shooting_skills          18944 non-null  int64  
 11  mentality                18944 non-null  int64  
 12  technical_skills         18944 non-null  int64  
 13  goalkeeping_ability      18944 non-null  float64
dtypes: float64(1), int64(1

In [14]:
df= df.astype(int)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   overall                  18944 non-null  int64
 1   potential                18944 non-null  int64
 2   age                      18944 non-null  int64
 3   passing                  18944 non-null  int64
 4   dribbling                18944 non-null  int64
 5   defending                18944 non-null  int64
 6   physic                   18944 non-null  int64
 7   attacking_crossing       18944 non-null  int64
 8   attacking_short_passing  18944 non-null  int64
 9   movement_reactions       18944 non-null  int64
 10  shooting_skills          18944 non-null  int64
 11  mentality                18944 non-null  int64
 12  technical_skills         18944 non-null  int64
 13  goalkeeping_ability      18944 non-null  int64
dtypes: int64(14)
memory usage: 2.0 MB


TRAINING

In [16]:
# extracting the target variable 'overall' (player's overall rating) and the feature variables from the DataFrame.
y = df['overall']  # target variable representing player's overall rating (Y values)
X = df.drop('overall', axis=1)  # feature variables excluding the 'overall' column (X values)

In [17]:
# importing the StandardScaler module from sklearn.preprocessing package.
from sklearn.preprocessing import StandardScaler

# initializing the StandardScaler object.
sc = StandardScaler()

In [18]:
# applying the StandardScaler to transform the feature variables (X) into a standardized form.
scaled = sc.fit_transform(X)


In [19]:
#creating a dataframe for feature variables
X=pd.DataFrame(scaled, columns=X.columns)

In [20]:
y.head()

0    92
1    92
2    91
3    91
4    91
Name: overall, dtype: int64

In [21]:
X.head()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   potential                18944 non-null  float64
 1   age                      18944 non-null  float64
 2   passing                  18944 non-null  float64
 3   dribbling                18944 non-null  float64
 4   defending                18944 non-null  float64
 5   physic                   18944 non-null  float64
 6   attacking_crossing       18944 non-null  float64
 7   attacking_short_passing  18944 non-null  float64
 8   movement_reactions       18944 non-null  float64
 9   shooting_skills          18944 non-null  float64
 10  mentality                18944 non-null  float64
 11  technical_skills         18944 non-null  float64
 12  goalkeeping_ability      18944 non-null  float64
dtypes: float64(13)
memory usage: 1.9 MB


In [22]:
y.value_counts()

65    1168
66    1139
64    1135
67    1112
63    1078
68     974
69     943
62     941
70     885
61     802
60     775
72     746
71     707
59     612
73     571
58     552
75     543
57     467
74     447
56     379
55     333
76     321
77     279
54     250
53     246
78     215
52     208
51     180
79     180
80     137
50     127
81      94
83      72
49      67
82      60
48      43
84      41
85      37
87      20
47      16
86      14
88      10
89       6
90       6
91       4
92       2
Name: overall, dtype: int64

In [23]:
# splitting the dataset into training and testing sets using train_test_split function.
# Xtrain: features for training, Xtest: features for testing, Ytrain: target values for training, Ytest: target values for testing.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# displaying the shape of the training feature set (Xtrain).
Xtrain.shape


(17049, 13)

**TRAINING**

USING RANDOM FOREST REGRESSOR

In [24]:
# importing the RandomForestRegressor module from the sklearn.ensemble package.
from sklearn.ensemble import RandomForestRegressor

In [25]:
# initializing the RandomForestRegressor model
rf=RandomForestRegressor()

In [26]:
# training the model using the training data (Xtrain and Ytrain)
rf.fit(Xtrain, Ytrain)

RandomForestRegressor()

In [27]:
# making predictions on the test data (Xtest) using the trained model.
y_pred=rf.predict(Xtest)
y_pred

array([51.51, 64.54, 70.  , ..., 69.99, 55.21, 65.39])

In [28]:
# calculating the mean absolute error between the predicted values (y_pred) and the actual values (Ytest).
mean_absolute_error(y_pred,Ytest)

0.7472664907651715

USING XGB_REGRESSOR

In [29]:
# importing the XGBRegressor module from the xgboost package.
from xgboost import XGBRegressor

In [30]:
# initializing the XGBRegressor model.
xgb_model = XGBRegressor()

In [31]:
# training the XGBRegressor model using the training data (Xtrain and Ytrain).
xgb_model.fit(Xtrain, Ytrain)

# making predictions on the test data (Xtest) using the trained XGBRegressor model.
y_pred_xgb = xgb_model.predict(Xtest)

# calculating the mean absolute error between the XGBRegressor predicted values (y_pred_xgb) and the actual values (Ytest).
mae_xgb = mean_absolute_error(y_pred_xgb, Ytest)
mae_xgb

0.7781703073305316

USING GRADIENT_BOOSTING_REGRESSOR

In [32]:
# importing the GradientBoostingRegressor module from the sklearn.ensemble package.
from sklearn.ensemble import GradientBoostingRegressor

# initializing the GradientBoostingRegressor model.
gb_model = GradientBoostingRegressor()

# training the GradientBoostingRegressor model using the training data (Xtrain and Ytrain).
gb_model.fit(Xtrain, Ytrain)

# making predictions on the test data (Xtest) using the trained GradientBoostingRegressor model.
y_pred_gb = gb_model.predict(Xtest)

# calculating the mean absolute error between the GradientBoostingRegressor predicted values (y_pred_gb) and the actual values (Ytest).
mae_gb = mean_absolute_error(y_pred_gb, Ytest)
mae_gb


1.001943178200062

CROSS-VALIDATION USING GRID SEARCH

In [33]:
from sklearn.model_selection import GridSearchCV


In [34]:
# these are the hyperparameters for Random Forest
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# these are the hyperparameters for XGBoost
xgb_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}


In [35]:
# performed a grid search with cross-validation to find the best RandomForestRegressor model using negative mean absolute error as the scoring metric.
grid_rf = GridSearchCV(estimator=RandomForestRegressor(),
                       param_grid=rf_params,
                       scoring='neg_mean_absolute_error',
                       cv=5)
grid_rf.fit(Xtrain, Ytrain)
best_rf = grid_rf.best_estimator_



In [36]:
# perform grid search with cross-validation to find the best XGBRegressor model using negative mean absolute error as the scoring metric.
grid_xgb = GridSearchCV(estimator=XGBRegressor(),
                        param_grid=xgb_params,
                        scoring='neg_mean_absolute_error',
                        cv=5)
grid_xgb.fit(Xtrain, Ytrain)
best_xgb = grid_xgb.best_estimator_



In [37]:
y_pred_best_rf = best_rf.predict(Xtest)
mae_best_rf = mean_absolute_error(y_pred_best_rf, Ytest)
print("Mean Absolute Error for Best Random Forest Model: ", mae_best_rf)

y_pred_best_xgb = best_xgb.predict(Xtest)
mae_best_xgb = mean_absolute_error(y_pred_best_xgb, Ytest)
print("Mean Absolute Error for Best XGBoost Model: ", mae_best_xgb)


Mean Absolute Error for Best Random Forest Model:  0.7330527704485489
Mean Absolute Error for Best XGBoost Model:  0.6915606637114271


ENSEMBLE MODEL USING ALL TWO MODELS (RF & XGB) ABOVE


In [38]:
#for the ensemble model we used only the RF and XGB because the gradient boosting regressor increase the mean absolute error and hence the ensemble model needed optimization
from sklearn.ensemble import VotingRegressor

ensemble_model = VotingRegressor(estimators=[('rf', rf), ('xgb', xgb_model)])

# Training the ensemble model
ensemble_model.fit(Xtrain, Ytrain)

# Making predictions using the ensemble model
ensemble_predictions = ensemble_model.predict(Xtest)

EVALUATION

In [39]:
# Calculating mean absolute error for the ensemble predictions
mae_ensemble = mean_absolute_error(ensemble_predictions, Ytest)
ensemble_predictions
Xtest
Ytest

print("Mean Absolute Error for Random Forest: ", mean_absolute_error(y_pred, Ytest))
print("Mean Absolute Error for XGBoost: ", mae_xgb)
print("Mean Absolute Error for Ensemble Model: ", mae_ensemble)

Mean Absolute Error for Random Forest:  0.7472664907651715
Mean Absolute Error for XGBoost:  0.7781703073305316
Mean Absolute Error for Ensemble Model:  0.7124758374458252


CONFIDENCE SCORE

In [40]:
# Assuming you have an array of predictions from K-fold cross-validation named 'predictions'
mean_prediction = np.mean(ensemble_predictions)
std_deviation = np.std(ensemble_predictions)


In [41]:
from scipy.stats import norm

# specified the confidence level
confidence_level = 0.95

# calculate the margin of error
z_score = norm.ppf((1 + confidence_level) / 2)  # Z-score for the given confidence level
margin_of_error = z_score * (std_deviation / np.sqrt(len(ensemble_predictions)))

# Calculate lower and upper bounds of the confidence interval
lower_bound = mean_prediction - margin_of_error
upper_bound = mean_prediction + margin_of_error
confidence_percentage = (1 - 2 * (1 - confidence_level)) * 100

print("Lower bound:", lower_bound)
print("Upper bound:", upper_bound)
print("Margin of error:", margin_of_error)
print("Confidence Percentage:", confidence_percentage)

Lower bound: 65.31982347410037
Upper bound: 65.93969122820589
Margin of error: 0.30993387705275666
Confidence Percentage: 89.99999999999999


TRAINING MODEL USING NEW DATASET (PLAYERS_22)

In [42]:
# Load the new dataset
new_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_22.csv', usecols=selected_columns)


**DATA PREPOCESSING**

In [43]:
new_df.fillna(0, inplace=True)
new_df['overall'] = new_df['overall'].apply(lambda x: 92 if x in [92, 93] else x)

**FEATURE ENGINEERING**

In [44]:
skill_attributes = ['skill_long_passing', 'skill_ball_control', 'skill_curve', 'skill_fk_accuracy', 'skill_dribbling']

new_df['technical_skills'] = new_df[skill_attributes].mean(axis=1)

new_df.drop(columns=skill_attributes, inplace=True)

new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  19239 non-null  int64  
 1   potential                19239 non-null  int64  
 2   age                      19239 non-null  int64  
 3   shooting                 19239 non-null  float64
 4   passing                  19239 non-null  float64
 5   dribbling                19239 non-null  float64
 6   defending                19239 non-null  float64
 7   physic                   19239 non-null  float64
 8   attacking_crossing       19239 non-null  int64  
 9   attacking_finishing      19239 non-null  int64  
 10  attacking_short_passing  19239 non-null  int64  
 11  attacking_volleys        19239 non-null  int64  
 12  movement_reactions       19239 non-null  int64  
 13  power_shot_power         19239 non-null  int64  
 14  power_long_shots      

In [45]:
shooting_attributes = ['shooting', 'power_shot_power', 'power_long_shots', 'attacking_volleys','attacking_finishing']

new_df['shooting_skills'] = new_df[shooting_attributes].mean(axis=1)

new_df.drop(columns=shooting_attributes, inplace=True)

new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  19239 non-null  int64  
 1   potential                19239 non-null  int64  
 2   age                      19239 non-null  int64  
 3   passing                  19239 non-null  float64
 4   dribbling                19239 non-null  float64
 5   defending                19239 non-null  float64
 6   physic                   19239 non-null  float64
 7   attacking_crossing       19239 non-null  int64  
 8   attacking_short_passing  19239 non-null  int64  
 9   movement_reactions       19239 non-null  int64  
 10  mentality_aggression     19239 non-null  int64  
 11  mentality_interceptions  19239 non-null  int64  
 12  mentality_positioning    19239 non-null  int64  
 13  mentality_vision         19239 non-null  int64  
 14  mentality_penalties   

In [46]:
mentality_attributes = ['mentality_aggression', 'mentality_interceptions', 'mentality_positioning',
                        'mentality_vision', 'mentality_penalties', 'mentality_composure']
imputer = SimpleImputer(strategy='mean')
new_df[mentality_attributes] = imputer.fit_transform(new_df[mentality_attributes])

new_df['mentality'] = new_df[mentality_attributes].mean(axis=1)

new_df.drop(columns=mentality_attributes, inplace=True)

new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  19239 non-null  int64  
 1   potential                19239 non-null  int64  
 2   age                      19239 non-null  int64  
 3   passing                  19239 non-null  float64
 4   dribbling                19239 non-null  float64
 5   defending                19239 non-null  float64
 6   physic                   19239 non-null  float64
 7   attacking_crossing       19239 non-null  int64  
 8   attacking_short_passing  19239 non-null  int64  
 9   movement_reactions       19239 non-null  int64  
 10  goalkeeping_diving       19239 non-null  int64  
 11  goalkeeping_handling     19239 non-null  int64  
 12  goalkeeping_kicking      19239 non-null  int64  
 13  goalkeeping_positioning  19239 non-null  int64  
 14  goalkeeping_reflexes  

In [47]:
goalkeeping_attributes = ['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
                          'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed']

imputer = SimpleImputer(strategy='mean')
new_df[goalkeeping_attributes] = imputer.fit_transform(new_df[goalkeeping_attributes])

new_df['goalkeeping_ability'] = new_df[goalkeeping_attributes].mean(axis=1)

new_df.drop(columns=goalkeeping_attributes, inplace=True)

new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  19239 non-null  int64  
 1   potential                19239 non-null  int64  
 2   age                      19239 non-null  int64  
 3   passing                  19239 non-null  float64
 4   dribbling                19239 non-null  float64
 5   defending                19239 non-null  float64
 6   physic                   19239 non-null  float64
 7   attacking_crossing       19239 non-null  int64  
 8   attacking_short_passing  19239 non-null  int64  
 9   movement_reactions       19239 non-null  int64  
 10  technical_skills         19239 non-null  float64
 11  shooting_skills          19239 non-null  float64
 12  mentality                19239 non-null  float64
 13  goalkeeping_ability      19239 non-null  float64
dtypes: float64(8), int64(6

**TRAINING AND EVALUATION**

In [48]:
y_new = new_df['overall']
X_new = new_df.drop('overall', axis=1)
X_new

,potential,age,passing,dribbling,defending,physic,attacking_crossing,attacking_short_passing,movement_reactions,technical_skills,shooting_skills,mentality,goalkeeping_ability
0,93,34,91.0,95.0,34.0,65.0,85,91,94,94.0,91.0,73.833333,9.000000
1,92,32,79.0,86.0,44.0,82.0,71,85,93,81.4,90.6,80.666667,8.500000
2,91,36,80.0,88.0,34.0,75.0,87,80,94,83.6,92.4,74.333333,9.666667
3,91,29,86.0,94.0,37.0,63.0,85,86,89,89.2,82.6,77.000000,9.833333
4,91,30,93.0,88.0,64.0,78.0,94,94,91,88.0,86.4,82.666667,9.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,52,22,46.0,48.0,42.0,49.0,46,50,53,42.6,34.6,42.333333,8.166667
19235,59,19,50.0,46.0,41.0,51.0,54,51,49,43.4,38.2,46.500000,7.833333
19236,55,21,45.0,49.0,41.0,52.0,39,49,46,43.8,38.0,42.500000,6.166667
19237,60,19,36.0,48.0,15.0,42.0,29,38,48,38.0,43.6,38.500000,8.833333


In [49]:
scaled_X_new = sc.fit_transform(X_new)
X_new = pd.DataFrame(scaled_X_new, columns=X_new.columns)

In [50]:
X_new_train,X_new_test,Y_new_train,Y_new_test=train_test_split(X_new,y_new,test_size=0.1,random_state=42,stratify = y_new)

In [51]:
rf.fit(X_new_train, Y_new_train)
y_pred_rf = rf.predict(X_new_test)


In [52]:
mean_absolute_error(y_pred_rf,Y_new_test)

0.7156340956340957

In [53]:
xgb_model.fit(X_new_train, Y_new_train)
y_pred_xgb = xgb_model.predict(X_new_test)
mae_xgb_new = mean_absolute_error(y_pred_xgb, Y_new_test)
mae_xgb_new

0.7446936292112989

**Hyperparameter Tuning for Random Forest Regression**

In [54]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters and their respective values to search through
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of trees in the forest
    'max_depth': [None, 10, 20],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# initialize Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# perform Grid Search on the training data
grid_search.fit(X_new_train, Y_new_train)

# Get the best hyperparameters found by Grid Search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# initialize a new Random Forest Regressor with the best hyperparameters
best_rf = RandomForestRegressor(**best_params)

# train the model with the best hyperparameters
best_rf.fit(X_new_train, Y_new_train)

# make predictions using the tuned model
y_pred_rf_tuned = best_rf.predict(X_new_test)

# Calculate mean absolute error for the tuned model
mae_rf_tuned = mean_absolute_error(y_pred_rf_tuned, Y_new_test)
print("Mean Absolute Error for Tuned Random Forest: ", mae_rf_tuned)


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Mean Absolute Error for Tuned Random Forest:  0.7126334026334026


**Hyperparameter Tuning for XGBoost Regression**

In [55]:
from sklearn.model_selection import GridSearchCV

# define the hyperparameters and their respective values to search through
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of boosting rounds
    'max_depth': [3, 6, 9],  # Maximum depth of the tree
    'learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage used to prevent overfitting
    'subsample': [0.8, 1.0],  # Fraction of samples used for fitting the trees
    'min_child_weight': [1, 5, 10]  # Minimum sum of instance weight (hessian) needed in a child
}

# initialize Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# perform Grid Search on the training data
grid_search.fit(X_new_train, Y_new_train)

# get the best hyperparameters found by Grid Search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# initialize a new XGBoost Regressor with the best hyperparameters
best_xgb = XGBRegressor(**best_params)

# train the model with the best hyperparameters
best_xgb.fit(X_new_train, Y_new_train)

# make predictions using the tuned model
y_pred_xgb_tuned = best_xgb.predict(X_new_test)

# calculate mean absolute error for the tuned model
mae_xgb_tuned = mean_absolute_error(y_pred_xgb_tuned, Y_new_test)
print("Mean Absolute Error for Tuned XGBoost: ", mae_xgb_tuned)


Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 9, 'min_child_weight': 10, 'n_estimators': 150, 'subsample': 0.8}
Mean Absolute Error for Tuned XGBoost:  0.6725658250202012


**TESTING USING THE ENSEMBLE MODEL**

In [56]:
from sklearn.ensemble import VotingRegressor

ensemble_model = VotingRegressor(estimators=[('rf', rf), ('xgb', xgb_model)])

ensemble_model.fit(Xtrain, Ytrain)

ensemble_predictions = ensemble_model.predict(Xtest)
print(ensemble_predictions)
print(Xtest)
print(Ytest)

mae_ensemble_new = mean_absolute_error(ensemble_predictions, Ytest)

mae_ensemble_new

[51.95428162 64.25309952 70.08895874 ... 70.0063945  55.5809964
 65.36623917]
       potential       age   passing  dribbling  defending    physic  \
18622   0.476817 -1.538316 -1.074897  -1.040132   0.238857  0.163729   
10987  -0.505208  0.164816  0.154589   0.249206   0.687310  0.840549   
4735   -0.177866  1.016381  0.351307   0.663636  -0.030216  0.569821   
2782    0.967830 -0.048076  0.056230   0.249206   1.225455  0.840549   
10844  -0.832549  0.590598  0.302128   0.295254   0.732156  0.614943   
...          ...       ...       ...        ...        ...       ...   
7651   -0.341537 -0.048076  0.744743   0.617588   0.642465  0.479579   
17840  -1.487233 -0.260967 -0.091308   0.065015   0.418238 -0.016756   
5106   -0.177866  0.590598  0.351307   0.939922  -0.747742 -0.242363   
17389  -0.505208 -0.899642  0.007051   0.018967  -0.119906 -0.106999   
8993   -0.341537 -0.048076  0.548025   0.525492   0.418238 -0.152120   

       attacking_crossing  attacking_short_passing  movem

0.7142978086043787

In [57]:
# saving the scaler object using joblib for later use in ensemble models
joblib.dump(sc, '/content/drive/My Drive/Colab Notebooks/scaler_ensemble.pkl')

['/content/drive/My Drive/Colab Notebooks/scaler_ensemble.pkl']

In [58]:
# save the ensemble model using pickle for future use
# pickle the ensemble model and save it to the specified file path
import pickle
filename = '/content/drive/My Drive/Colab Notebooks/sports_prediction_ensemble_model.pkl'
pickle.dump(ensemble_model, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))